In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException,TimeoutException,WebDriverException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from time import sleep
from bs4 import BeautifulSoup

### Cars 24 Scrapper Function

In [29]:
def cars24_scrapper(url):
    
    driver = webdriver.Chrome("chromedriver.exe")
    driver.get(url)

    driver.maximize_window()

    items = []

    last_height = driver.execute_script("return document.body.scrollHeight")

    itemTargetCount = 1000

    while itemTargetCount > len(items):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        sleep(1)
        new_height = driver.execute_script("return document.body.scrollHeight")

        if new_height == last_height:
            break

        last_height = new_height

        elements = driver.find_elements(By.XPATH,'/html/body/div[1]/div[2]/div/div[3]/div/div/div[@class="col-4"]')

        items = elements
        
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    headings = soup.findAll('h2',attrs={'class':"_3FpCg"})
    
    man_year, brand , model = [] , [] , []
    for heading in headings:
        values = heading.text.split(" ")
        man_year.append(values[0])
        brand.append(values[1])
        model.append(" ".join(values[2:]))
        
    variants = soup.findAll('p',attrs={'class':"cvakB"})
    
    variants_details , automatic_or_not = [], []
    for variant in variants:
        text = variant.text
        text = text.lower()
        if " manual" in text:
            automatic_or_not.append("Manual")
            variants_details.append(text.replace(" manual",""))
        else:
            automatic_or_not.append("Automatic")
            variants_details.append(text.replace(" automatic",""))
    
    details = soup.findAll('ul',attrs={'class':"bVR0c"})
    
    driven_km , fuel , owner = [] , [] , []
    for detail in details:
        list_item = detail.findAll("li")
        list_item_text = [li.text for li in list_item]
        driven_km.append(list_item_text[0])
        fuel.append(list_item_text[2])
        owner.append(list_item_text[1])
        
    prices = soup.findAll('div',attrs={'class':"_7udZZ"})
    prices = [price.text for price in prices]
    
    df = pd.DataFrame({"Brand":brand , 
                   "Model":model  , 
                   "Variant":variants_details , 
                   "Manufacturing Year":man_year , 
                   "Automatic/Manual":automatic_or_not ,
                   "Distance Driven":driven_km,
                   "Fuel":fuel,
                   "Owner":owner,
                   "Prices":prices})
    
    return df

### Fetching Data of All Cities

In [6]:
links = ["https://www.cars24.com/buy-used-cars-delhi-ncr/", 
         "https://www.cars24.com/buy-used-cars-hyderabad/",
         "https://www.cars24.com/buy-used-cars-jaipur/",
         "https://www.cars24.com/buy-used-cars-kochi/",
         "https://www.cars24.com/buy-used-cars-lucknow/",
         "https://www.cars24.com/buy-used-cars-noida/",
         "https://www.cars24.com/buy-used-cars-gurgaon/",
         "https://www.cars24.com/buy-used-cars-mumbai/",
         "https://www.cars24.com/buy-used-cars-pune/",
         "https://www.cars24.com/buy-used-cars-bangalore/",
         "https://www.cars24.com/buy-used-cars-chennai/",
         "https://www.cars24.com/buy-used-cars-trivandrum/",
         "https://www.cars24.com/buy-used-cars-kolkata/", 
         "https://www.cars24.com/buy-used-cars-ahmedabad/",
         "https://www.cars24.com/buy-used-cars-chandigarh/",
         "https://www.cars24.com/buy-used-cars-ludhiana/",
         "https://www.cars24.com/buy-used-cars-surat/",
         "https://www.cars24.com/buy-used-cars-patna/",
         "https://www.cars24.com/buy-used-cars-nashik/"]

list_of_data = []
for link in links:
    list_of_data.append(cars24_scrapper(link))

C:\Users\User\AppData\Local\Temp/ipykernel_1396/4186929771.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("chromedriver.exe")


### Combining All Data In Single DataFrame

In [7]:
data = pd.concat(list_of_data, axis=0, ignore_index=True)

In [8]:
data

,Brand,Model,Variant,Manufacturing Year,Automatic/Manual,Distance Driven,Fuel,Owner
0,Renault,Kwid,rxl,2016,Manual,"33,139 km",Petrol,2nd Owner
1,KIA,SELTOS,htk 1.5 petrol,2020,Manual,"4,765 km",Petrol,1st Owner
2,Maruti,Swift Dzire,vxi,2010,Manual,"29,766 km",Petrol,1st Owner
3,KIA,SELTOS,htk 1.5 petrol,2020,Manual,"11,146 km",Petrol,1st Owner
4,Maruti,Swift Dzire,vxi 1.3,2009,Manual,"26,597 km",Petrol,2nd Owner
...,...,...,...,...,...,...,...,...
5446,Maruti,Swift Dzire,vxi,2013,Manual,"38,304 km",Petrol,1st Owner
5447,Hyundai,Xcent,sx 1.2 opt,2015,Manual,"68,125 km",Petrol,1st Owner
5448,Maruti,Swift,zdi amt,2018,Automatic,"89,777 km",Diesel,1st Owner
5449,Maruti,Wagon R Stingray,vxi,2013,Manual,"60,664 km",Petrol,3rd Owner


In [15]:
df=pd.DataFrame(data)

In [16]:
df

,Brand,Model,Variant,Manufacturing Year,Automatic/Manual,Distance Driven,Fuel,Owner
0,Renault,Kwid,rxl,2016,Manual,"33,139 km",Petrol,2nd Owner
1,KIA,SELTOS,htk 1.5 petrol,2020,Manual,"4,765 km",Petrol,1st Owner
2,Maruti,Swift Dzire,vxi,2010,Manual,"29,766 km",Petrol,1st Owner
3,KIA,SELTOS,htk 1.5 petrol,2020,Manual,"11,146 km",Petrol,1st Owner
4,Maruti,Swift Dzire,vxi 1.3,2009,Manual,"26,597 km",Petrol,2nd Owner
...,...,...,...,...,...,...,...,...
5446,Maruti,Swift Dzire,vxi,2013,Manual,"38,304 km",Petrol,1st Owner
5447,Hyundai,Xcent,sx 1.2 opt,2015,Manual,"68,125 km",Petrol,1st Owner
5448,Maruti,Swift,zdi amt,2018,Automatic,"89,777 km",Diesel,1st Owner
5449,Maruti,Wagon R Stingray,vxi,2013,Manual,"60,664 km",Petrol,3rd Owner


In [28]:
used_car=df.to_csv('used_cars.csv')